In [10]:
import numpy as np
from mlp import DenseLayer, MLP

np.random.seed(0)

def mse(y_true: np.ndarray, y_pred: np.ndarray) -> np.float64:
    return np.mean(np.square(y_true - y_pred))

In [11]:
l1 = 1
l2 = []
l3 = 5
seq = [l1, *l2, l3]
print(seq)

[1, 5]


In [12]:
# Input data
rows = 5
cols = 10
input_vector = np.random.normal(size=(rows, cols))
dummy_pred_vector = np.random.normal(size=(rows, cols))

In [13]:
# Instantiate Layer
lyr = DenseLayer(input_dims=input_vector.shape[-1], num_units=2)

In [14]:
# Layer call
lyr_activations, lyr_wt_inputs = lyr(input_vector)
print('input:', input_vector.shape)
print('output:', lyr_activations.shape, lyr_wt_inputs.shape)
print(lyr_activations, '\n\n', lyr_wt_inputs)

input: (5, 10)
output: (5, 2) (5, 2)
[[-0.42647349 -0.24779502]
 [-0.28689659 -0.40105034]
 [-0.32231351 -0.45992227]
 [-0.18448811  0.24800787]
 [ 0.17294692  0.21348298]] 

 [[-0.42647349 -0.24779502]
 [-0.28689659 -0.40105034]
 [-0.32231351 -0.45992227]
 [-0.18448811  0.24800787]
 [ 0.17294692  0.21348298]]


In [21]:
model = MLP(input_dims=input_vector.shape[-1], hidden_units=2, targets=1, loss_function=mse, learning_rate=1e-3, l_layers=2)

3


In [20]:
# Result of forward pass
fwd = model._forward_pass(inputs=input_vector)
print(fwd)
print(fwd.shape)

[[ 0.13224191]
 [ 0.01783743]
 [-0.16562248]
 [-0.15245786]
 [-0.05169993]]
(5, 1)


In [ ]:
batch_size = 32
samples = 100
y_batch_indices = np.random.choice(
            a=samples, size=(samples//batch_size, batch_size), replace=False) 

3
[[22 51 90 17 94 76 89 87  8 53 98 55 50 31 26 28 88 79 34 74 39 85 12 16
  83 64 20 67 58 77 25  7]
 [86 36 21 72 84 57  9 11 96 82 68 49  5 14 15 37 19 33 69 78 59 56  4 91
  75  6 95 44 65 62 29 40]
 [71 24 81 41 73 45 13 23 42 52 92 38 63 60  2 46 70 93 32  3  0 80 97 99
   1 30 48 27 47 43 18 10]]
